## Integración de base de datos

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida28/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida28/doctorado/ssev-analytics


In [2]:
import pandas as pd

from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime

In [3]:
import warnings
warnings.filterwarnings('ignore')

Parque

In [4]:
park = 'cerro_saroche'

DIR = f'./{park}/'

Data NARX precipitación

In [5]:
pd_narx_precipitacion = pd.read_pickle(f'{DIR}data/narx_precipitacion.pkl')
pd_narx_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.072351,0.108708,508.541046
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,0.299087,508.541046
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,2.657068,508.541046
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,1.260912,508.541046
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,1.177589,508.541046


Data ANN precipitación

In [6]:
pd_ann_precipitacion = pd.read_pickle(f'{DIR}data/ann_precipitacion.pkl')
pd_ann_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann,ndvi_media
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.072351,0.108708,508.541046,0.108708,0.207416,NaN
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,0.299087,508.541046,0.299087,0.262076,NaN
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,2.657068,508.541046,2.657068,0.698307,NaN
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,1.260912,508.541046,1.260912,1.442033,NaN
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,1.177589,508.541046,1.177589,1.287615,NaN


Data NARX NDVI

In [7]:
pd_narx_ndvi = pd.read_pickle(f'{DIR}data/narx_ndvi.pkl')
pd_narx_ndvi.periodo = pd.to_datetime(pd_narx_ndvi.periodo)
pd_narx_ndvi.head()

,id_point,periodo,type,ndvi_prediction,ndvi_media
0,1,2013-02-01,training,0.497152,0.499354
1,1,2013-03-01,training,0.394080,0.423398
2,1,2013-04-01,training,0.453817,0.441592
3,1,2013-05-01,training,0.528053,0.634015
4,1,2013-06-01,training,0.571129,0.582212


Integración de base de datos

In [8]:
pd_analisis = pd.merge(pd_ann_precipitacion[['park',
                                            'periodo',
                                            'year',
                                            'month',
                                            'id_point',
                                            'latitud',
                                            'longitud',
                                            'type',
                                            'precipitacion_mm',
                                            'precipitacion_narx',
                                            'prediction_ann',
                                            'elevacion_media']],
                        pd_narx_ndvi[['id_point', 'periodo','ndvi_prediction', 'ndvi_media']],
                        on=['id_point','periodo'],
                        how='left')

pd_analisis['time'] = pd_analisis['periodo'].apply(lambda x: datetime.toordinal(x))
pd_analisis['time_actualizacion'] = int(datetime.today().toordinal())
pd_analisis = pd_analisis[['park',
                        'year',
                        'month',
                        'id_point',
                        'latitud',
                        'longitud',
                        'type',
                        'precipitacion_mm',
                        'precipitacion_narx',
                        'prediction_ann',
                        'elevacion_media',
                        'ndvi_prediction',
                        'ndvi_media',
                        'time',
                        'time_actualizacion']]
pd_analisis.head()

,park,year,month,id_point,latitud,longitud,type,precipitacion_mm,precipitacion_narx,prediction_ann,elevacion_media,ndvi_prediction,ndvi_media,time,time_actualizacion
0,cerro_saroche,1995,1,1,10.31,-69.83,training,0.072351,0.108708,0.207416,508.541046,NaN,NaN,728294,738506
1,cerro_saroche,1995,2,1,10.31,-69.83,training,0.340843,0.299087,0.262076,508.541046,NaN,NaN,728325,738506
2,cerro_saroche,1995,3,1,10.31,-69.83,training,2.290730,2.657068,0.698307,508.541046,NaN,NaN,728353,738506
3,cerro_saroche,1995,4,1,10.31,-69.83,training,1.064486,1.260912,1.442033,508.541046,NaN,NaN,728384,738506
4,cerro_saroche,1995,5,1,10.31,-69.83,training,1.114330,1.177589,1.287615,508.541046,NaN,NaN,728414,738506


In [9]:
pd_analisis.type.unique()

array(['training', 'test', 'prediction'], dtype=object)

In [10]:
import json

json_data = '{ "data":' + pd_analisis.to_json(orient="records") +\
                ',"park" : "terepaima"' + "}"

with open(f'{DIR}data/json_data.json', 'w') as outfile:
    outfile.write(json_data)

In [11]:
# Mongo coleccion
documentos = json.loads( pd_analisis.to_json(orient="records") )

documentos[-1]

{'park': 'cerro_saroche',
 'year': 2023,
 'month': 5,
 'id_point': 9,
 'latitud': 10.21,
 'longitud': -69.53,
 'type': 'prediction',
 'precipitacion_mm': None,
 'precipitacion_narx': 2.465892701,
 'prediction_ann': 1.4775212163,
 'elevacion_media': 769.9250488281,
 'ndvi_prediction': 0.3697563107,
 'ndvi_media': None,
 'time': 738641,
 'time_actualizacion': 738506}

Conexión Mongo DB

In [12]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological',
 'meteorological',
 'estimateSSTNino34',
 'polygons',
 'SSTNino34']

In [13]:
# insertando coleccion
coleccion = db['estimateMeteorological']

In [14]:
# Insertando documentos
for doc in documentos:
  coleccion.update_one({"time":doc.get('time'),
                        "id_point":doc.get('id_point'),
                        "park":"cerro_saroche"}, {"$set":doc}, upsert=True)